 ## 1. Lectura de datos

In [1]:
!pip install nltk
import nltk

from nltk import download

download('punkt_tab')                           # Tokenización
nltk.download('averaged_perceptron_tagger')     # POS tagging
nltk.download('averaged_perceptron_tagger_eng') # POS tagging
nltk.download('wordnet')                        # WordNet lemmatizer
nltk.download('omw-1.4')                        # WordNet multilingüe

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.data import path
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
from nltk.corpus import wordnet as wn
import numpy as np

path.append(".")

In [3]:
!pip install contractions
import contractions

In [4]:
import csv
import pandas as pd
from bs4 import BeautifulSoup
from pprint import pprint
import re
from bs4 import MarkupResemblesLocatorWarning
import warnings

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import recall_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.neighbors import KNeighborsClassifier
import spacy

In [6]:
import json
import os
import shutil
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
from collections import defaultdict
from tempfile import mkdtemp

!pip install whoosh
from whoosh.index import create_in
from whoosh.index import open_dir
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser
from whoosh.analysis import StemmingAnalyzer
from whoosh import scoring
from whoosh.query import Term, Or

In [7]:
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

In [8]:
palabras_vacias_ingles = stopwords.words('english')

In [9]:
nlp = spacy.load("en_core_web_sm")

In [10]:
def elimina_html(contenido):
    return BeautifulSoup(contenido).get_text()

def elimina_no_alfanumerico(contenido):
    return [re.sub(r'[^\w]', '', palabra)
            for palabra in contenido
            if re.search(r'\w', palabra)]

def expandir_constracciones(contenido):
    return contractions.fix(contenido)

def pasar_a_minuscula(contenido):
    return contenido.lower()

def limpiar_texto(texto):
    texto = re.sub(r'[^a-zA-Z\s]', ' ', texto)  # Reemplaza todo lo que no es letra o espacio con espacio
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def elimina_palabras_vacias(contenido):
    return [palabra for palabra in contenido if palabra not in palabras_vacias_ingles]

def lematizador(contenido):
    lemmatizer = WordNetLemmatizer()
    pos_tags = pos_tag(contenido)

    resultado = []
    for palabra, tag in pos_tags:
        if tag.startswith('VB'):  # Verbos
            resultado.append(lemmatizer.lemmatize(palabra, pos='v'))  # infinitivo
        else:  # Sustantivos y el resto tal como están
            resultado.append(palabra)

    return resultado

def extraer_noun_chunks(tokens):
    resultados = []
    doc = nlp(" ".join(tokens))

    noun_chunks = [chunk.text.lower().strip() for chunk in doc.noun_chunks if len(chunk.text.split()) <= 3]
    noun_chunks_set = set(noun_chunks)

    i = 0
    while i < len(tokens):
        composed2 = " ".join(tokens[i:i+2]).lower()
 
        if composed2 in noun_chunks_set:
            i += 2 
        else:
            resultados.append(tokens[i].lower())
            i += 1

    return resultados + noun_chunks


In [11]:
def proceso_contenido(texto):
    texto = elimina_html(texto)
    texto = expandir_constracciones(texto)
    texto = pasar_a_minuscula(texto)
    texto = limpiar_texto(texto)                # Limpiar antes de tokenizar
    tokens = word_tokenize(texto)               
    tokens = elimina_no_alfanumerico(tokens)    # Limpiar tokens individuales
    tokens = elimina_palabras_vacias(tokens)
    tokens = lematizador(tokens)
    return tokens

 ## 2. Construcción de ambos sistemas de recuperación de la información

In [12]:
def procesar_csv_corpus(ruta_csv):
    """
    Carga y procesa un archivo CSV con tres columnas: autor, título y cuerpo.
    Aplica la función proceso_contenido a cada parte y devuelve una lista de listas procesadas.
    
    Parámetros:
        ruta_csv (str): Ruta al archivo CSV. Por defecto es "news_corpus.csv".
    
    Retorna:
        list: Lista donde cada elemento es una lista combinada del contenido procesado.
    """
    # Cargar CSV con codificación segura y punto y coma como separador
    df = pd.read_csv(ruta_csv, encoding="latin-1", sep=";", quotechar='"')

    # Verificar que tenga exactamente 3 columnas
    assert df.shape[1] == 3, "El CSV no tiene exactamente 3 columnas. Revisa el separador o las comillas."

    resultados = []

    for _, fila in df.iterrows():
        autor = fila.iloc[0]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]

        autor_proc = proceso_contenido(autor)
        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada = autor_proc + titulo_proc + cuerpo_proc
        resultados.append(fila_combinada)
    
    return resultados

In [13]:
def documentos_corpus_cuerpo(ruta_csv):
    # Cargar CSV con codificación segura y punto y coma como separador
    df = pd.read_csv(ruta_csv, encoding="latin-1", sep=";", quotechar='"')

    # Verificar que tenga exactamente 3 columnas
    assert df.shape[1] == 3, "El CSV no tiene exactamente 3 columnas. Revisa el separador o las comillas."

    resultados = []

    for _, fila in df.iterrows():
        cuerpo = fila.iloc[2]
        resultados.append(cuerpo)

    return resultados

In [14]:
def lectura_normalizada_corpus():
    df = pd.read_csv("news_corpus.csv", encoding="latin-1", sep=";", quotechar='"')
    resultados = []

    for index, fila in df.iterrows():
        autor = [fila.iloc[0]]
        titulo = fila.iloc[1]
        cuerpo = fila.iloc[2]   

        titulo_proc = proceso_contenido(titulo)
        cuerpo_proc = proceso_contenido(cuerpo)

        # Unir las tres listas en una sola lista combinada
        fila_combinada =  autor + titulo_proc + cuerpo_proc

        contenido_final = extraer_noun_chunks(fila_combinada)
        resultados.append(contenido_final)
    return resultados

def expand_term(term):
    related = set()
    for syn in wn.synsets(term):
        for lemma in syn.lemmas():
            word = lemma.name().replace('_', ' ').lower()
            if word != term:
                related.add(word)
    return related

def expand_corpus_with_synonyms(documento):
    doc_counter = Counter(documento)
    expanded_doc = []
    for word, count in doc_counter.items():
        # Añadimos la palabra original tantas veces como aparece
        expanded_doc.extend([word] * count)
        # Obtenemos sinónimos y también los añadimos con la misma frecuencia
        synonyms = expand_term(word)
        for syn in synonyms:
            expanded_doc.extend([syn] * count)
    return expanded_doc


def crear_indice_whoosh(corpus_normalizado):
    if os.path.exists("indice_whoosh"):
        shutil.rmtree("indice_whoosh")  # Elimina índice anterior si existe
    os.mkdir("indice_whoosh")

    schema = Schema(id=ID(stored=True, unique=True), contenido=TEXT(stored=True))
    ix = create_in("indice_whoosh", schema)
    writer = ix.writer()

    for i, doc in enumerate(corpus_normalizado):
        contenido = " ".join(doc)
        writer.add_document(id=str(i), contenido=contenido)
    
    writer.commit()


def buscar_con_whoosh(tokens_query, corpus_normalizado):
    ix = open_dir("indice_whoosh")  # Abrir índice Whoosh previamente creado

    with ix.searcher() as searcher:
        # Crear la consulta OR con todos los términos
        terms = [Term("contenido", token) for token in tokens_query]
        query = Or(terms)

        resultados = searcher.search(query, limit=None)

        # Crear diccionario: clave = doc_id, valor = documento procesado
        docs_encontrados = []
        indices_docs = []
        for hit in resultados:
            doc_id = int(hit['id'])
            docs_encontrados.append(corpus_normalizado[doc_id])
            indices_docs.append(doc_id)

        return docs_encontrados, indices_docs


In [15]:
def crear_indice_whoosh_para_corpus(subcorpus, ruta_indice="indice"):
    """
    Crea un índice Whoosh para un subcorpus dado y lo guarda en disco.

    Parámetros:
        subcorpus (list of list of str): Lista de documentos tokenizados.
        ruta_indice (str): Carpeta donde se almacenará el índice Whoosh.
        offset (int): Valor entero que se suma al índice para generar el doc_id.
    """
    schema = Schema(doc_id=ID(stored=True),
                    content=TEXT(stored=True, analyzer=StemmingAnalyzer()))

    if not os.path.exists(ruta_indice):
        os.mkdir(ruta_indice)

    ix = create_in(ruta_indice, schema)
    writer = ix.writer()

    for i, doc in enumerate(subcorpus):
        joined_content = ' '.join(doc)
        writer.add_document(doc_id=str(i), content=joined_content)

    writer.commit()

In [16]:
def lectura_documento(documento):
    documento_procesado = proceso_contenido(documento) 
    contenido_final = extraer_noun_chunks(documento_procesado)
    return contenido_final

In [17]:
def tfidf_del_documento(documento_normalizado, vectorizer):
    # Convertir el documento (lista de tokens) a string
    texto = " ".join(documento_normalizado)
    # Transformar usando el vectorizador ya entrenado
    X_doc = vectorizer.transform([texto])  # devuelve matriz sparse 1xN
    return X_doc

In [18]:
def tfidf_por_corpus(corpus_normalizado):
    # Convertimos el corpus a lista de strings
    texts = [" ".join(doc) for doc in corpus_normalizado]

    # Vectorizador TF-IDF (1 y 2-gramas)
    vectorizer = TfidfVectorizer(ngram_range=(1, 2))
    X = vectorizer.fit_transform(texts)  # Matriz TF-IDF sparse
    terms = vectorizer.get_feature_names_out()

    # Retornamos la matriz y vocabulario (términos)
    return X, terms, vectorizer

In [19]:
def similitud_coseno(tfidf_corpus, tfidf_doc, indices, umbral=0.0):
    # Calcular similitud coseno entre documento y corpus
    similitudes = cosine_similarity(tfidf_doc, tfidf_corpus).flatten()

    # Filtrar documentos que superan el umbral
    indices_filtrados = [i for i, sim in enumerate(similitudes) if sim > umbral]

    # Ordenar índices por similitud descendente
    indices_ordenados = sorted(indices_filtrados, key=lambda i: similitudes[i], reverse=True)

    # Devolver lista de (indice, similitud)
    return [(indices[i], similitudes[i]) for i in indices_ordenados]


In [28]:
def ranking_documentos_por_consulta(consulta, ruta_csv):
    corpus_normalizado = lectura_normalizada_corpus()
    documentos = documentos_corpus_cuerpo(ruta_csv)
    
    crear_indice_whoosh(corpus_normalizado)

    lectura = lectura_documento(consulta)
    docs_relevantes, indices_docs = buscar_con_whoosh(lectura, corpus_normalizado)
    tfidf_corpus, terms, vectorizer = tfidf_por_corpus(docs_relevantes)
    tfidf_documento = tfidf_del_documento(lectura, vectorizer)
    resultados_similitud = similitud_coseno(tfidf_corpus, tfidf_documento, indices_docs)

    for idx, score in resultados_similitud:
        print("-" * 60)
        print(f"Documento {idx}: {documentos[idx]}")


In [26]:
def documentos_consultas_booleanas(consulta, ruta_csv, indice_path="indice"):
    documentos_encontrados = set()
    documentos_procesados = procesar_csv_corpus(ruta_csv)
    documentos = documentos_corpus_cuerpo(ruta_csv)
    
    crear_indice_whoosh_para_corpus(documentos_procesados)
    
    ix = open_dir(indice_path)
    parser = QueryParser("content", ix.schema)

    with ix.searcher(weighting=scoring.BM25F()) as searcher:
        try:
            query = parser.parse(consulta)
            resultados = searcher.search(query, limit=None)
            documentos_encontrados = set([r['doc_id'] for r in resultados])
        except Exception as e:
            print(f"Error al procesar la consulta '{consulta}': {e}")

    if documentos_encontrados:
        for idx_str in sorted(documentos_encontrados):
            idx = int(idx_str)
            print("-" * 60)
            print(f"Documento {idx}: {documentos[idx]}")
    else:
        print("No se han encontrado documentos relacionados con su consulta")

## 3. Input para cada uno de los sistemas de recuperación de la información

In [27]:
# Input para sistema de recuperación de la información específico para consultas según el modelo booleano

ruta_csv = "news_corpus.csv"

# Solicitar consulta al usuario
consulta = input("Introduce tu consulta: ")

# Ejecutar documentos
documentos_consultas_booleanas(consulta, ruta_csv)

Introduce tu consulta: Trump
------------------------------------------------------------
Documento 100: There is only a 5% chance that the Earth will avoid warming by at least 2C come the end of the century, according to new research that paints a sobering picture of the international effort to stem dangerous climate change.Global trends in the economy, emissions and population growth make it extremely unlikely that the planet will remain below the 2C threshold set out in the Paris climate agreement in 2015, the study states. The Paris accord, signed by 195 countries, commits to holding the average global temperature to ?well below 2C? above pre-industrial levels and sets a more aspirational goal to limit warming to 1.5C. This latter target is barely plausible, the new research finds, with just a 1% chance that temperatures will rise by less than 1.5C. ?We?re closer to the margin than we think,? said Adrian Raftery, a University of Washington academic who led the research, published i

In [29]:
# Input para sistema de recuperación de la información específico para consultas de texto libre

ruta_csv = "news_corpus.csv"

# Solicitar consulta al usuario
consulta = input("Introduce tu consulta: ")

# Ejecutar ranking
ranking_documentos_por_consulta(consulta, ruta_csv)

Introduce tu consulta: News about Donald Trump
------------------------------------------------------------
Documento 134: Donald Trump personally dictated the press statement issued in the name of his eldest son Donald Jr that misleadingly downplayed the significance of a 2016 meeting with a Kremlin-linked Russian lawyer, a new report alleged on Monday night.According to the Washington Post, Trump personally intervened to prevent senior White House advisers from issuing a full and truthful account of the meeting on 9 June 2016 in which Donald Trump Jr, the president?s son-in-law Jared Kushner, and then presidential campaign manager Paul Manafort came face-to-face with four Russians. One of the Russian visitors was the well-connected lawyer Natalia Veselnitskaya.The report, based on multiple though largely anonymous sources that included the president?s own advisers, has the potential to cause political, and even legal, trouble for the White House because it draws Trump himself much cl